In [ ]:
import pandas as pd
import requests

ROW_LOOKUP = {
    'CountryWiki': 'Country',
    'Born': 'Born',
    'Height': 'Height',
    'TurnedPro': 'Turned.*pro',
    'Retired': 'Retired',
    'Coach': 'Coach',
    'EarningsWiki': 'Prize.*money',
    'CareerRecord': 'Career.*record',
    'Titles': 'Career.*titles',
    'HighRank': 'Highest.*ranking',
    }

BAD_URLS = []

PLAYERS_INFO = {}

In [ ]:
urls = [
    #'https://en.wikipedia.org/wiki/Asdf_Xyz',
    'https://en.wikipedia.org/wiki/Serena_Williams',
    #'https://en.wikipedia.org/wiki/Venus_Williams',
    'https://en.wikipedia.org/wiki/Simona_Halep',
    #'https://en.wikipedia.org/wiki/Maria_Sharapova',
    #'https://en.wikipedia.org/wiki/Caroline_Wozniacki',
    #'https://en.wikipedia.org/wiki/Victoria_Azarenka',
    #'https://en.wikipedia.org/wiki/Petra_Kvitova',
    #'https://en.wikipedia.org/wiki/Angelique_Kerber',
    #'https://en.wikipedia.org/wiki/Agnieszka_Radwanska',
    #'https://en.wikipedia.org/wiki/Svetlana_Kuznetsova',
    #'https://en.wikipedia.org/wiki/Martina_Hingis',
    #'https://en.wikipedia.org/wiki/Garbine_Muguruza',
    #'https://en.wikipedia.org/wiki/Kim_Clijsters',
    #'https://en.wikipedia.org/wiki/Karolina_Pliskova',
    #'https://en.wikipedia.org/wiki/Ashleigh_Barty',
    #'https://en.wikipedia.org/wiki/Lindsay_Davenport',
    #'https://en.wikipedia.org/wiki/Steffi_Graf',
    #'https://en.wikipedia.org/wiki/Martina_Navratilova',
    #'https://en.wikipedia.org/wiki/Elina_Svitolina',
    #'https://en.wikipedia.org/wiki/Naomi_Osaka',
    #'https://en.wikipedia.org/wiki/Justine_Henin',
    #'https://en.wikipedia.org/wiki/Samantha_Stosur',
    #'https://en.wikipedia.org/wiki/Jelena_Jankovic',
    #'https://en.wikipedia.org/wiki/Iga_Swiatek',
    #'https://en.wikipedia.org/wiki/Cori_Gauff',
    #'https://en.wikipedia.org/wiki/Emma_Raducanu',
    'https://en.wikipedia.org/wiki/Qiang_Wang',
    'https://en.wikipedia.org/wiki/Anna_Kournikova',
    'https://en.wikipedia.org/wiki/Denise_Starr',
    'https://en.wikipedia.org/wiki/Kai-Chen_Chang',
    'https://en.wikipedia.org/wiki/Marie-Gayanay_Mikaelian',
    ]

def get_urls_from_list():
    for url in urls:
        yield url

The table needs to be navigated by looping through the data frame rows and the looping needs to abort once the value 'Doubles' is encountered.  Otherwise, looping by lookup of the Attr names returns dataframes if the same attribute is found multiple times.

OK, got the proper subset of the data table (sub_tab), but still having problem with extracting the cell values.
What next?
Need the following to extract a value of the cell:  
sub_tab.loc[sub_tab.index[4], 'Value']

In [ ]:
def find_data_items(sub_tab):
    info = {}
    for key in ROW_LOOKUP.keys():
        #print(key)
        val = ROW_LOOKUP[key]
        #print(val)
        if len(sub_tab[sub_tab['Attr'].str.contains(val, regex=True, na=False)]) > 0:
            row_idx = sub_tab[sub_tab['Attr'].str.contains(val, regex=True, na=False)].index[0]
            #print(sub_tab.loc[sub_tab.index[row_idx], 'Value'])
            info[key] = sub_tab.loc[sub_tab.index[row_idx], 'Value']

    return info

In [ ]:
def get_singles_subtable(table):
    doubles_idx = table.index[table['Attr'] == 'Doubles'].to_list()
    #print(doubles_idx)
    if len(doubles_idx) > 0:
        sub_tab = table[:doubles_idx[0]]
        return sub_tab
    else:
        return table

In [ ]:
# using requests.get as a workaround in pandas read_html bug which doesn't work with Unicode
# https://github.com/pandas-dev/pandas/issues/21499

def get_wiki_info(url):
    try:
        r = requests.get(url)
        page = r.text
        tables = pd.read_html(page, attrs={'class': 'infobox vcard'})
    except:
        print('Failed to read url ' + url)
        BAD_URLS.append(url)
        return

    table = tables[0]
    (rows, cols) = table.shape
    #print(table.shape)
    
    columns = ['Attr', 'Value']
    for i in range(2, cols):
        columns.append('Extra_' + str(i))
    table.columns = columns
    
    sub_tab = get_singles_subtable(table)
    return find_data_items(sub_tab)

In [ ]:
def get_urls_from_df():
    df = pd.read_pickle('./Data/earnings.pickle')
    for i in range(0, len(df)):
        url = df.loc[df.index[i], 'WikiLink']
        yield url

In [ ]:
from random import random
from time import sleep

#for url in get_urls_from_list():
for url in get_urls_from_df():
    print("processing: " + url)
    r = random()
    sleep(r)
    PLAYERS_INFO[url] = get_wiki_info(url)


In [ ]:
pi = pd.DataFrame(PLAYERS_INFO).transpose()
pi

In [ ]:
pi.to_html('./Data/player_info.html', render_links=True)

In [ ]:
pi.to_pickle('./Data/player_info.pickle')

In [ ]:
import pickle

print(len(BAD_URLS))
file = open('./Data/bad_urls.pickle', 'wb')

# dump information to that file
pickle.dump(BAD_URLS, file)

# close the file
file.close()
